# Installs & functions

In [ ]:
%pip install qiskit
%pip install qiskit-aer
import numpy as np
import random
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 33.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 61.6 MB/s eta 0:00:00


In [ ]:
def makeBell(qc):
  # Bell state phi+ (correlated, not anti correlated (bob and alice will get same padding))
  qc.h(aliceRegister)
  qc.cx(aliceRegister, bobRegister)

def aliceMeasure(qci, i):
  if i == 0:
    # Alice measures in "+" (H/V)
    qci.measure(aliceRegister, cr[0])
  else:
    # Alice measures in "x" (D/A)
    qci.h(aliceRegister)
    qci.measure(aliceRegister, cr[0])

def eveMeasure(qci, i):
  if i == 0:
    # Eve measures Bob's bit in "+" (H/V)
    qci.measure(bobRegister, cr[2])
  else:
    # Eve measures Bob's bit in "x" (D/A)
    qci.h(bobRegister)
    qci.measure(bobRegister, cr[2])
    qci.h(bobRegister)

def bobMeasure(qci, i):
  if i == 0:
    # Bob measures in "+" (H/V)
    qci.measure(bobRegister, cr[1])
  else:
    # Bob measures in "x" (D/A)
    qci.h(bobRegister)
    qci.measure(bobRegister, cr[1])

# Actual code

In [ ]:
# Variables
n = 1000
eve_present = 0 # 0 no, 1 yes

In [ ]:
aliceBases = ""
eveBases = ""
bobBases = ""
aliceMeasurements = ""
eveMeasurements = ""
bobMeasurements = ""

for i in range(n):
  # Make circuit
  aliceRegister = QuantumRegister(1) # Alice has qr[0], Bob has qr[1]
  bobRegister = QuantumRegister(1)
  cr = ClassicalRegister(3)
  qc = QuantumCircuit(aliceRegister, bobRegister, cr)
  makeBell(qc)

  # Measure
  a = np.random.randint(0, 2)
  aliceBases += str(a)
  b = np.random.randint(0, 2)
  bobBases += str(b)
  aliceMeasure(qc, a)

  if eve_present == 1:
    e = np.random.randint(0, 2)
    eveBases += str(e)
    eveMeasure(qc, e)

  bobMeasure(qc, b)

  # Get actual values
  backend = Aer.get_backend('qasm_simulator')
  counts= backend.run(qc, shots=1).result().get_counts(qc)
  bitstring = list(counts.keys())[0]
  if eve_present == 1:
    eveMeasurements += bitstring[0]
  aliceMeasurements += bitstring[2]
  bobMeasurements += bitstring[1]

print("Alice's bases:" + str(aliceBases))
print("Eve's bases  :" + str(eveBases))
print("Bob's bases  :" + str(bobBases))
print(aliceMeasurements)
print(eveMeasurements)
print(bobMeasurements)

Alice's bases:00010010100100101010010101110110110100100110110010101110001110111101010101011101010010110101100110110000000011110001010110100111110101111011111011010000101111010100101011001100001101111101001110101101000000111101100000010010111101111110000101101101111100010111011111001011000101111001101001101011101000101111101101101100000100000010110100001111110101111011111011010111110100000110000100000101011111010010101010011010110001101110110110110110000011010100000011100000010010000010000000011010110010101110101110111011000101100100111111110001110101011101100001111111101101100100101101010001111111011101011111110100101100000101010011110010000111001110010110000110011011110111001000001101110001001000000110010001111010001100000100111111111001111110111011111100001001110101111001101011001111100111100100010001110011100000100001010011101010011101110001111011000000100000110011000110000100010010000111101000110011101101010011011010000100011101001011000000000111010011110001110100000101010101100011

In [ ]:
sharedKeyA = ""
sharedKeyB = ""

for i in range(n):
  if aliceBases[i] == bobBases[i]:
    sharedKeyA += aliceMeasurements[i]
    sharedKeyB += bobMeasurements[i]

if sharedKeyA != sharedKeyB:
  print("Gotcha eve")
elif eve_present == 1:
  print("Eve was hidden")
else:
  print("Eve not present")
  print("eve_present: " + str(eve_present))

print(sharedKeyA)
print(sharedKeyB)

Eve not present
eve_present: 0
0011010101001111001000101010000100011011110011000000100010010001111011110110101011001010001001101011111000100000001011000001110011100010101001010111110011000111100001011010100011110100011100100011000001101111100100111100000011011001100100111010100101001111010100110001000001111010011001011011011100001010110000010011111010101100101110001011101101011001011100101001011101011110011110000110100101010110011011110011101110101010001110110001101000010001100011001110110011001000110101100010011111011001010101101110001
0011010101001111001000101010000100011011110011000000100010010001111011110110101011001010001001101011111000100000001011000001110011100010101001010111110011000111100001011010100011110100011100100011000001101111100100111100000011011001100100111010100101001111010100110001000001111010011001011011011100001010110000010011111010101100101110001011101101011001011100101001011101011110011110000110100101010110011011110011101110101010001110110001101000010001100011001